In [ ]:
test_mode = False

In [24]:
import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv
import datetime
#from datetime import timezone

warnings.filterwarnings("ignore")

In [3]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2024-12-30 11:26:58,850 INFO: Initializing external client
2024-12-30 11:26:58,850 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 11:27:00,335 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [4]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60


today


1735550821

In [5]:

# Step 2: Retrieve the feature group by name and version
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

# Step 3: Read the data from the feature group
s_data_df = solana_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
solana_max_date = s_data_df['timestamp'].max()
solana_difference = (today - solana_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {solana_max_date}")
print(f"Difference in days: {solana_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
The maximum date is: 1735344000
Difference in days: 2


In [6]:

# Step 2: Retrieve the feature group by name and version
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)

# Step 3: Read the data from the feature group
b_data_df = bitcoin_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
bitcoin_max_date = b_data_df['timestamp'].max()
bitcoin_difference = (today - bitcoin_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {bitcoin_max_date}")
print(f"Difference in days: {bitcoin_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 
The maximum date is: 1735344000
Difference in days: 2


In [7]:
#fs = project.get_feature_store(name='kth_lab1_2024_featurestore')
fs = project.get_feature_store()

fng_feature_group = fs.get_feature_group(
    name='f_n_g_index', 
    version=8)
fng_df = fng_feature_group.read()

# Step 4: Find the maximum date
fng_max_date = int(fng_df['timestamp'].max())
fng_difference = (today - fng_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {fng_max_date}")
print(f"Difference in days: {fng_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.91s) 
The maximum date is: 1735344000
Difference in days: 2


### Getting missing data

### Request solana price from last available data till now

In [8]:

url = "https://data-api.cryptocompare.com//spot/v1/historical/days"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "SOL-USD",  
    "limit": solana_difference,  
    "api_key": crypto_api_key,
}
if solana_difference!=0:
    sol_response = trigger_request(url, params)
    sol_response_df = pd.DataFrame(sol_response["Data"])
    sol_response_df = sol_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    sol_response_df.columns = sol_response_df.columns.str.lower()
    print("Num entries to add: ", len(sol_response_df))
else:
    sol_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

Num entries to add:  2


In [9]:
sol_response_df.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,195.15,197.19,188.26,189.60,120179.830193,45590.791614,74589.038579
1,1735516800,189.60,193.80,188.77,192.18,50341.494122,28207.816437,22133.677685


### Request bitcoin price from last available data till now

In [10]:
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "BTC-USD",  
    "limit": bitcoin_difference,  
    "api_key": crypto_api_key,
}

if bitcoin_difference>0:
    bitcoin_response = trigger_request(url, params)
    bitcoin_response_df = pd.DataFrame(bitcoin_response["Data"])
    bitcoin_response_df = bitcoin_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    bitcoin_response_df.columns = bitcoin_response_df.columns.str.lower()
    print("Num entries to add: ", len(bitcoin_response_df))
else:
    bitcoin_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

bitcoin_response_df.head()

Num entries to add:  2


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,95204.1,95204.1,92860.7,93551.0,698.547127,229.720452,468.826675
1,1735516800,93551.0,94000.0,93033.0,93567.1,239.316127,91.377367,147.938760


In [11]:
bitcoin_response_df

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,95204.1,95204.1,92860.7,93551.0,698.547127,229.720452,468.826675
1,1735516800,93551.0,94000.0,93033.0,93567.1,239.316127,91.377367,147.938760


### Request fear and greed index values from last available data till now

In [22]:
url = "https://api.alternative.me/fng/?"

# Set the parameters for the API request
params = {
    "limit": fng_difference  # Number of days of data to retrieve
}

if fng_difference>0:
    fng_response = trigger_request(url, params)
    fng_df = pd.DataFrame(fng_response['data'])
    fng_df['timestamp'] = fng_df['timestamp']#.astype(float)
    #fng_df.rename(columns={'value_classification': 'fng_classification'}, inplace=True)
    #fng_df.rename(columns={'value': 'fng_value'}, inplace=True)

    fng_df.drop(columns=["time_until_update"],inplace=True)
    fng_df.head()
else:
    fng_df = pd.DataFrame(columns=["fng_value","fng_classification","timestamp"])
fng_df.head()

,value,value_classification,timestamp
0,65,Greed,1735516800
1,73,Greed,1735430400


### Now we need to use the responses to update the data in Hopsworks

In [19]:
if not test_mode and sol_response_df.shape[0]>0:
    solana_feature_group.insert(sol_response_df)



Uploading Dataframe: 100.00% |██████████| Rows 2/2 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_1_offline_fg_materialization/executions


In [20]:
if not test_mode and bitcoin_response_df.shape[0]>0:
    bitcoin_feature_group.insert(bitcoin_response_df)


Uploading Dataframe: 100.00% |██████████| Rows 2/2 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bitcoin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/bitcoin_1_offline_fg_materialization/executions


In [23]:
if not test_mode and fng_df.shape[0]>0:
    fng_feature_group.insert(fng_df)

Uploading Dataframe: 100.00% |██████████| Rows 2/2 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: f_n_g_index_8_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/f_n_g_index_8_offline_fg_materialization/executions
